In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('DNN-EdgeIIoT-dataset.csv')

/tmp/ipykernel_24749/2769549640.py:1: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('DNN-EdgeIIoT-dataset.csv')


In [3]:
df

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,2021 11:44:10.081753000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
1,2021 11:44:10.162218000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,4.0,MQTT,0,0.0,4.0,0.0,0.0,0.0,0,Normal
2,2021 11:44:10.162271000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
3,2021 11:44:10.162641000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
4,2021 11:44:10.166132000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,Temperature_and_Humidity,24.0,0.0,0.0,0.0,0.0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219196,2021 23:24:32.816050000,166.75.162.225,192.168.0.128,0,0.0,0.0,0,31814.0,45620.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP
2219197,2021 23:24:32.816595000,70.162.34.183,192.168.0.128,0,0.0,0.0,0,27718.0,45636.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP
2219198,2021 23:24:32.818043000,40.13.95.244,192.168.0.128,0,0.0,0.0,0,18502.0,45672.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP
2219199,2021 23:24:32.820831000,18.132.75.125,192.168.0.128,0,0.0,0.0,0,1862.0,45737.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP


## Preprocess

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Assuming df is your pandas DataFrame
# Separate features and target variable

df = df.sample(frac=1, random_state=42).reset_index(drop=True)
midpoint = len(df) // 2
df = df.iloc[:midpoint, :]

for col in df.columns:
    
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
df = df.fillna(0)

X = df.drop(columns=['Attack_label','Attack_type'])
y = df['Attack_label']
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.LongTensor(y_val.values).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create data loaders
batch_size = 64  # You can adjust this
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

/opt/conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1069: RuntimeWarning: overflow encountered in square
  temp **= 2
/opt/conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/conda/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:83: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

class DNN(nn.Module):
    def __init__(self, input_size):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 2)  # Assuming binary classification for Attack_label

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

# Determine the input size for the first layer of the network
input_size = X_train_tensor.shape[1]

# Instantiate the model
dnn_model = DNN(input_size)

dnn_model.to(device)

DNN(
  (layer1): Linear(in_features=61, out_features=128, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=2, bias=True)
)

In [6]:
print(X_train_tensor.device)
print(y_train_tensor.device)

cuda:0
cuda:0


In [7]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(dnn_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # You can adjust this
for epoch in range(num_epochs):
    dnn_model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = dnn_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Optionally, you can add a validation step here

print('Training completed')


Epoch [1/10], Loss: nan
Epoch [2/10], Loss: nan
Epoch [3/10], Loss: nan
Epoch [4/10], Loss: nan
Epoch [5/10], Loss: nan
Epoch [6/10], Loss: nan
Epoch [7/10], Loss: nan
Epoch [8/10], Loss: nan
Epoch [9/10], Loss: nan
Epoch [10/10], Loss: nan
Training completed


In [8]:
import numpy as np
# Ensure the model is in evaluation mode
dnn_model.eval()

# No gradient is needed for evaluation
with torch.no_grad():
    y_pred = []
    y_true = []

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = dnn_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Convert lists to numpy arrays for metric calculation
y_pred = np.array(y_pred)
y_true = np.array(y_true)

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Calculating TPR, FPR, FNR, TNR
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
fnr = fn / (tp + fn)
tnr = tn / (tn + fp)

# Other metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)  # Same as TPR
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

# Print the metrics
print(f"Accuracy of DNN: {accuracy:.4f}")
print(f"Precision of DNN: {precision:.4f}")
print(f"Recall (TPR) of DNN: {recall:.4f}")
print(f"F1 Score of DNN: {f1:.4f}")
print(f"AUC of DNN: {auc:.4f}")
print(f"FPR of DNN: {fpr:.4f}")
print(f"FNR of DNN: {fnr:.4f}")
print(f"TNR of DNN: {tnr:.4f}")

Accuracy of DNN: 0.7294
Precision of DNN: 0.0000
Recall (TPR) of DNN: 0.0000
F1 Score of DNN: 0.0000
AUC of DNN: 0.5000
FPR of DNN: 0.0000
FNR of DNN: 1.0000
TNR of DNN: 1.0000


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
all_preds = []
all_true = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to('cuda')
        outputs = dnn_model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_true.extend(labels.cpu().numpy())
# Convert to numpy arrays
all_predictions = np.array(all_preds)
all_labels = np.array(all_true)

In [11]:
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.7294
Precision: 0.5320
Recall: 0.7294
F1 Score: 0.6152


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7293709444844989
Precision: 0.36468547224224945
Recall: 0.5
F1 Score: 0.4217550588615412


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='micro')
recall = recall_score(all_labels, all_predictions, average='micro')
f1 = f1_score(all_labels, all_predictions, average='micro')

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7293709444844989
Precision: 0.7293709444844989
Recall: 0.7293709444844989
F1 Score: 0.7293709444844989


In [14]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(all_true, all_preds)
num_classes = 2
total_TPR, total_FPR, total_FNR, total_TNR = 0, 0, 0, 0

for i in range(num_classes):
    TP = conf_matrix[i, i]
    FP = np.sum(conf_matrix[:, i]) - TP
    FN = np.sum(conf_matrix[i, :]) - TP
    TN = np.sum(conf_matrix) - TP - FP - FN

    TPR = TP / (TP + FN) if TP + FN > 0 else 0
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    FNR = FN / (FN + TP) if FN + TP > 0 else 0
    TNR = TN / (TN + FP) if TN + FP > 0 else 0

    total_TPR += TPR
    total_FPR += FPR
    total_FNR += FNR
    total_TNR += TNR

macro_TPR = total_TPR / num_classes
macro_FPR = total_FPR / num_classes
macro_FNR = total_FNR / num_classes
macro_TNR = total_TNR / num_classes

macro_metrics = {
    "Macro True Positive Rate (TPR)": round(macro_TPR, 4),
    "Macro False Positive Rate (FPR)": round(macro_FPR, 4),
    "Macro False Negative Rate (FNR)": round(macro_FNR, 4),
    "Macro True Negative Rate (TNR)": round(macro_TNR, 4)
}

macro_metrics

{'Macro True Positive Rate (TPR)': 0.5,
 'Macro False Positive Rate (FPR)': 0.5,
 'Macro False Negative Rate (FNR)': 0.5,
 'Macro True Negative Rate (TNR)': 0.5}

In [15]:
num_classes = 2
total_TP, total_FP, total_FN, total_TN = 0, 0, 0, 0

for i in range(num_classes):
    TP = conf_matrix[i, i]
    FP = np.sum(conf_matrix[:, i]) - TP
    FN = np.sum(conf_matrix[i, :]) - TP
    TN = np.sum(conf_matrix) - TP - FP - FN

    total_TP += TP
    total_FP += FP
    total_FN += FN
    total_TN += TN

micro_TPR = total_TP / (total_TP + total_FN) if total_TP + total_FN > 0 else 0
micro_FPR = total_FP / (total_FP + total_TN) if total_FP + total_TN > 0 else 0
micro_FNR = total_FN / (total_FN + total_TP) if total_FN + total_TP > 0 else 0
micro_TNR = total_TN / (total_TN + total_FP) if total_TN + total_FP > 0 else 0

micro_metrics = {
    "Micro True Positive Rate (TPR)": round(micro_TPR, 4),
    "Micro False Positive Rate (FPR)": round(micro_FPR, 4),
    "Micro False Negative Rate (FNR)": round(micro_FNR, 4),
    "Micro True Negative Rate (TNR)": round(micro_TNR, 4)
}

micro_metrics

{'Micro True Positive Rate (TPR)': 0.7294,
 'Micro False Positive Rate (FPR)': 0.2706,
 'Micro False Negative Rate (FNR)': 0.2706,
 'Micro True Negative Rate (TNR)': 0.7294}

In [16]:
num_classes = 2
weighted_TPR, weighted_FPR, weighted_FNR, weighted_TNR = 0, 0, 0, 0
total_instances = np.sum(conf_matrix)

for i in range(num_classes):
    TP = conf_matrix[i, i]
    FP = np.sum(conf_matrix[:, i]) - TP
    FN = np.sum(conf_matrix[i, :]) - TP
    TN = np.sum(conf_matrix) - TP - FP - FN

    TPR = TP / (TP + FN) if TP + FN > 0 else 0
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    FNR = FN / (FN + TP) if FN + TP > 0 else 0
    TNR = TN / (TN + FP) if TN + FP > 0 else 0

    class_weight = np.sum(conf_matrix[i, :]) / total_instances

    weighted_TPR += TPR * class_weight
    weighted_FPR += FPR * class_weight
    weighted_FNR += FNR * class_weight
    weighted_TNR += TNR * class_weight

weighted_metrics = {
    "Weighted True Positive Rate (TPR)": round(weighted_TPR, 4),
    "Weighted False Positive Rate (FPR)": round(weighted_FPR, 4),
    "Weighted False Negative Rate (FNR)": round(weighted_FNR, 4),
    "Weighted True Negative Rate (TNR)": round(weighted_TNR, 4)
}

weighted_metrics

{'Weighted True Positive Rate (TPR)': 0.7294,
 'Weighted False Positive Rate (FPR)': 0.7294,
 'Weighted False Negative Rate (FNR)': 0.2706,
 'Weighted True Negative Rate (TNR)': 0.2706}

## Attack

In [17]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from torchvision import transforms

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(dnn_model.parameters(), lr=0.001)
input_shape = (df.shape[1],)
# Wrap the model
classifier = PyTorchClassifier(
    model=dnn_model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape = input_shape,
    nb_classes=2,
)

In [20]:
from sklearn.metrics import precision_recall_fscore_support
# Ensure X_test and y_test are correctly prepared before this point
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test_indices = np.argmax(y_test, axis=1)
else:
    y_test_indices = y_test
X_test = X_test.astype('float32')

epsilon_values = [0.01, 0.1]

for epsilon in epsilon_values:
    # Craft adversarial samples with FGSM
    adv_crafter = FastGradientMethod(classifier, eps=epsilon)
    x_test_adv = adv_crafter.generate(x=X_test)

    # Save the adversarial examples to a file
    filename = f'x_test_adv_FGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)
    
    # Evaluate the classifier on the adversarial examples
    preds_FGSM = np.argmax(classifier.predict(x_test_adv), axis=1)

   # Calculate precision, recall, and F1 score
    precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
        y_test_indices, preds_FGSM, average='micro')
    precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
        y_test_indices, preds_FGSM, average='macro')

    # Calculate accuracy for reference
    acc = np.sum(preds_FGSM == y_test_indices) / y_test_indices.shape[0]

    # Calculate confusion matrix and then TPR, TNR, FPR, FNR for each class
    conf_matrix = confusion_matrix(y_test_indices, preds_FGSM)
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)

    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)

    # Averaging TPR, TNR, FPR, FNR
    TPR_avg = np.mean(TPR)
    TNR_avg = np.mean(TNR)
    FPR_avg = np.mean(FPR)
    FNR_avg = np.mean(FNR)
    TP_micro = np.sum(TP)
    FP_micro = np.sum(FP)
    FN_micro = np.sum(FN)
    TN_micro = np.sum(TN)

    # Micro averages for applicable metrics
    TPR_micro = TP_micro / (TP_micro + FN_micro)
    FPR_micro = FP_micro / (FP_micro + TN_micro)

    
    # Print the results for each epsilon
    print(f"Epsilon: {epsilon}")
    print(f"Accuracy: {acc}")
    print(f"Micro Precision: {precision_micro:.2f}")
    print(f"Macro Precision: {precision_macro:.2f}")
    print(f"Micro Recall: {recall_micro:.2f}")
    print(f"Macro Recall: {recall_macro:.2f}")
    print(f"Micro F1 Score: {fscore_micro:.2f}")
    print(f"Macro F1 Score: {fscore_macro:.2f}")
    print(f"Average TNR: {TNR_avg:.2f}, Average FPR: {FPR_avg:.2f}, Average FNR: {FNR_avg:.2f}\n")
    print(f"Macro TNR: {TNR_avg:.2f}, Macro FNR: {FNR_avg:.2f},Macro FPR: {FPR_avg:.2f}")
    print()


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epsilon: 0.01
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50

Epsilon: 0.1
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from art.attacks.evasion import ProjectedGradientDescent
eps_step_value = 0.01  
max_iter_value = 10 
# Convert the one-hot encoded labels to class indices if necessary
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test_indices = np.argmax(y_test, axis=1)
else:
    y_test_indices = y_test  # If y_test is already class indices

# Iterate over epsilon values
for epsilon in epsilon_values:
    # Craft adversarial samples with PGD
    adv_crafter = ProjectedGradientDescent(
        classifier,
        eps=epsilon,
        eps_step=eps_step_value,
        max_iter=max_iter_value
    )
    x_test_adv = adv_crafter.generate(x=X_test)

    # Evaluate the classifier on the adversarial examples
    preds_PDG = np.argmax(classifier.predict(x_test_adv), axis=1)

    filename = f'x_test_adv_PGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)
    
    # Calculate micro and macro precision, recall, and F1 score
    precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
        y_test_indices, preds_PDG, average='micro')
    precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
        y_test_indices, preds_PDG, average='macro')
    
        # Calculate accuracy for reference
    acc = np.sum(preds_FGSM == y_test_indices) / y_test_indices.shape[0]
    
    # Calculate confusion matrix and then TPR, TNR, FPR, FNR for each class
    conf_matrix = confusion_matrix(y_test_indices, preds_PDG)
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)

    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)

    # Averaging TPR, TNR, FPR, FNR
    TPR_avg = np.mean(TPR)
    TNR_avg = np.mean(TNR)
    FPR_avg = np.mean(FPR)
    FNR_avg = np.mean(FNR)
    TP_micro = np.sum(TP)
    FP_micro = np.sum(FP)
    FN_micro = np.sum(FN)
    TN_micro = np.sum(TN)

    # Micro averages for applicable metrics
    TPR_micro = TP_micro / (TP_micro + FN_micro)
    FPR_micro = FP_micro / (FP_micro + TN_micro)

    
    # Print the results for each epsilon
    print(f"Epsilon: {epsilon}")
    print(f"Accuracy: {acc}")
    print(f"Micro Precision: {precision_micro:.2f}")
    print(f"Macro Precision: {precision_macro:.2f}")
    print(f"Micro Recall: {recall_micro:.2f}")
    print(f"Macro Recall: {recall_macro:.2f}")
    print(f"Micro F1 Score: {fscore_micro:.2f}")
    print(f"Macro F1 Score: {fscore_macro:.2f}")
    print(f"Average TNR: {TNR_avg:.2f}, Average FPR: {FPR_avg:.2f}, Average FNR: {FNR_avg:.2f}\n")
    print(f"Macro TNR: {TNR_avg:.2f}, Macro FNR: {FNR_avg:.2f},Macro FPR: {FPR_avg:.2f}")
    print()

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epsilon: 0.01
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



Epsilon: 0.1
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from art.attacks.evasion import BasicIterativeMethod
eps_step_value = 0.01  
max_iter_value = 10 
# Convert the one-hot encoded labels to class indices if necessary
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test_indices = np.argmax(y_test, axis=1)
else:
    y_test_indices = y_test  # If y_test is already class indices

# Iterate over epsilon values
for epsilon in epsilon_values:
    # Craft adversarial samples with PGD
    adv_crafter = BasicIterativeMethod(
        classifier,
        eps=epsilon,
    )
    x_test_adv = adv_crafter.generate(x=X_test)

    # Evaluate the classifier on the adversarial examples
    preds_BIM = np.argmax(classifier.predict(x_test_adv), axis=1)
    
    filename = f'x_test_adv_BIM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)
    
    # Calculate micro and macro precision, recall, and F1 score
    precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
        y_test_indices, preds_BIM, average='micro')
    precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
        y_test_indices, preds_BIM, average='macro')
        # Calculate accuracy for reference
    acc = np.sum(preds_FGSM == y_test_indices) / y_test_indices.shape[0]
    # Calculate confusion matrix and then TPR, TNR, FPR, FNR for each class
    conf_matrix = confusion_matrix(y_test_indices, preds_BIM)
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)

    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)

    # Averaging TPR, TNR, FPR, FNR
    TPR_avg = np.mean(TPR)
    TNR_avg = np.mean(TNR)
    FPR_avg = np.mean(FPR)
    FNR_avg = np.mean(FNR)
    TP_micro = np.sum(TP)
    FP_micro = np.sum(FP)
    FN_micro = np.sum(FN)
    TN_micro = np.sum(TN)

    # Micro averages for applicable metrics
    TPR_micro = TP_micro / (TP_micro + FN_micro)
    FPR_micro = FP_micro / (FP_micro + TN_micro)

    
    # Print the results for each epsilon
    print(f"Epsilon: {epsilon}")
    print(f"Accuracy: {acc}")
    print(f"Micro Precision: {precision_micro:.2f}")
    print(f"Macro Precision: {precision_macro:.2f}")
    print(f"Micro Recall: {recall_micro:.2f}")
    print(f"Macro Recall: {recall_macro:.2f}")
    print(f"Micro F1 Score: {fscore_micro:.2f}")
    print(f"Macro F1 Score: {fscore_macro:.2f}")
    print(f"Average TNR: {TNR_avg:.2f}, Average FPR: {FPR_avg:.2f}, Average FNR: {FNR_avg:.2f}\n")
    print(f"Macro TNR: {TNR_avg:.2f}, Macro FNR: {FNR_avg:.2f},Macro FPR: {FPR_avg:.2f}")
    print()

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epsilon: 0.01
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



Epsilon: 0.1
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
from art.attacks.evasion import DeepFool

eps_step_value = 0.01  
max_iter_value = 10 
# Convert the one-hot encoded labels to class indices if necessary
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test_indices = np.argmax(y_test, axis=1)
else:
    y_test_indices = y_test  # If y_test is already class indices

# Iterate over epsilon values
for epsilon in epsilon_values:
    # Craft adversarial samples with PGD
    adv_crafter = DeepFool(
        classifier,
        epsilon=epsilon,
        batch_size=128
    )
    x_test_adv = adv_crafter.generate(x=X_test)
    
    # Evaluate the classifier on the adversarial examples
    preds_DF = np.argmax(classifier.predict(x_test_adv), axis=1)
    filename = f'x_test_adv_DF_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)
    
    # Calculate micro and macro precision, recall, and F1 score
    precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
        y_test_indices, preds_DF, average='micro')
    precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
        y_test_indices, preds_DF, average='macro')
        # Calculate accuracy for reference
    acc = np.sum(preds_DF == y_test_indices) / y_test_indices.shape[0]
    # Calculate confusion matrix and then TPR, TNR, FPR, FNR for each class
    conf_matrix = confusion_matrix(y_test_indices, preds_DF)
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)

    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)

    # Averaging TPR, TNR, FPR, FNR
    TPR_avg = np.mean(TPR)
    TNR_avg = np.mean(TNR)
    FPR_avg = np.mean(FPR)
    FNR_avg = np.mean(FNR)
    TP_micro = np.sum(TP)
    FP_micro = np.sum(FP)
    FN_micro = np.sum(FN)
    TN_micro = np.sum(TN)

    # Micro averages for applicable metrics
    TPR_micro = TP_micro / (TP_micro + FN_micro)
    FPR_micro = FP_micro / (FP_micro + TN_micro)

    
    # Print the results for each epsilon
    print(f"Epsilon: {epsilon}")
    print(f"Accuracy: {acc}")
    print(f"Micro Precision: {precision_micro:.2f}")
    print(f"Macro Precision: {precision_macro:.2f}")
    print(f"Micro Recall: {recall_micro:.2f}")
    print(f"Macro Recall: {recall_macro:.2f}")
    print(f"Micro F1 Score: {fscore_micro:.2f}")
    print(f"Macro F1 Score: {fscore_macro:.2f}")
    print(f"Average TNR: {TNR_avg:.2f}, Average FPR: {FPR_avg:.2f}, Average FNR: {FNR_avg:.2f}\n")
    print(f"Macro TNR: {TNR_avg:.2f}, Macro FNR: {FNR_avg:.2f},Macro FPR: {FPR_avg:.2f}")
    print()

DeepFool: 100%|██████████| 1734/1734 [10:10<00:00,  2.84it/s]
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epsilon: 0.01
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



DeepFool: 100%|██████████| 1734/1734 [09:49<00:00,  2.94it/s]


Epsilon: 0.1
Accuracy: 0.7293709444844989
Micro Precision: 0.73
Macro Precision: 0.36
Micro Recall: 0.73
Macro Recall: 0.50
Micro F1 Score: 0.73
Macro F1 Score: 0.42
Average TNR: 0.50, Average FPR: 0.50, Average FNR: 0.50

Macro TNR: 0.50, Macro FNR: 0.50,Macro FPR: 0.50



/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
